In [ ]:
#!/usr/bin/env python3

import requests
import json
import base64
import time

def test_mcp_server_with_llabs_payload():
    """Test MCP server with exact 11Labs payload format"""
    
    # MCP server details
    mcp_url = 'https://matrosmcp.duckdns.org/topdesk/mcp'
    token = 'e3707c16425c14fa417e2384a12748c0c7c51dfdfd1714c58992215983f33257'
    
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json',
        'Accept': 'application/json, text/event-stream'
    }
    
    print("🧪 Testing MCP server with exact 11Labs payload format")
    print("=" * 60)
    
    # Initialize MCP server
    print("🔧 Initializing MCP server...")
    init_data = {
        'jsonrpc': '2.0',
        'id': int(time.time() * 1000),
        'method': 'initialize',
        'params': {
            'protocolVersion': '2024-11-05',
            'capabilities': {},
            'clientInfo': {
                'name': 'test-client',
                'version': '1.0.0'
            }
        }
    }
    
    response = requests.post(mcp_url, headers=headers, json=init_data, timeout=30)
    print(f"Init Status: {response.status_code}")
    
    if response.status_code != 200:
        print(f"❌ Init failed: {response.text}")
        return
    
    session_id = response.headers.get('mcp-session-id')
    print(f"Session ID: {session_id}")
    
    if not session_id:
        print("❌ No session ID")
        return
    
    headers['mcp-session-id'] = session_id
    time.sleep(2)
    
    # Test with exact 11Labs payload
    print("\n🎯 Testing with exact 11Labs payload...")
    
    llabs_payload = {
        "incident_fields": {
            "priority_name": "P1 (I&A)",
            "request": "Medewerker Jacob (5421) kan niet inloggen op Windows. Foutmelding: 'geen gebruiker onder deze naam'. Meerdere collega's hebben hetzelfde probleem. Locatie: Den Haag, gebouw A, afdeling 3. Staat volledig stil.",
            "caller_phone_number": "06-123456789",
            "category_name": "Core applicaties",
            "brief_description": "Kan niet inloggen op Windows"
        },
        "caller_id": "5421"
    }
    
    print(f"Payload: {json.dumps(llabs_payload, indent=2)}")
    
    # Call the tool
    tool_call_data = {
        'jsonrpc': '2.0',
        'id': int(time.time() * 1000),
        'method': 'tools/call',
        'params': {
            'name': 'topdesk_create_incident',
            'arguments': llabs_payload
        }
    }
    
    print(f"\nTool call: {json.dumps(tool_call_data, indent=2)}")
    
    tool_response = requests.post(mcp_url, headers=headers, json=tool_call_data, timeout=30)
    print(f"\nTool Status: {tool_response.status_code}")
    print(f"Tool Response: {tool_response.text}")
    
    # Parse SSE response
    if tool_response.status_code == 200:
        response_text = tool_response.text
        if 'data: ' in response_text:
            json_part = response_text.split('data: ')[1].strip()
            try:
                result = json.loads(json_part)
                print(f"\nParsed result: {json.dumps(result, indent=2)}")
                
                if 'result' in result:
                    print(f"\n✅ SUCCESS! Tool call worked:")
                    print(f"Result: {result['result']}")
                elif 'error' in result:
                    print(f"\n❌ Tool call error: {result['error']}")
                else:
                    print(f"\nUnexpected response: {result}")
            except json.JSONDecodeError as e:
                print(f"\n❌ JSON parse error: {e}")
                print(f"Raw JSON: {json_part}")
        else:
            print(f"\n❌ No data field in response: {response_text}")
    else:
        print(f"\n❌ HTTP error: {tool_response.status_code}")
        print(f"Response: {tool_response.text}")

def test_direct_topdesk_api():
    """Test direct TOPdesk API for comparison"""
    
    print("\n" + "=" * 60)
    print("🔍 Testing direct TOPdesk API for comparison")
    print("=" * 60)
    
    # TOPdesk API details
    base_url = "https://pietervanforeest.topdesk.net/tas/api"
    username = "API_AIPILOTS"
    password = "g2jjr-kp6oc-tct6p-ie237-fyub5"
    
    # Build auth token
    api_token = base64.b64encode(f"{username}:{password}".encode()).decode()
    
    headers = {
        'Authorization': f'Basic {api_token}',
        'Content-Type': 'application/json'
    }
    
    # Convert 11Labs payload to TOPdesk format
    topdesk_payload = {
        "briefDescription": "Kan niet inloggen op Windows",
        "request": "Medewerker Jacob (5421) kan niet inloggen op Windows. Foutmelding: 'geen gebruiker onder deze naam'. Meerdere collega's hebben hetzelfde probleem. Locatie: Den Haag, gebouw A, afdeling 3. Staat volledig stil.",
        "caller": {
            "id": "5421"
        },
        "category": {
            "name": "Core applicaties"
        },
        "priority": {
            "name": "P1 (I&A)"
        }
    }
    
    print(f"TOPdesk payload: {json.dumps(topdesk_payload, indent=2)}")
    
    # Test with TOPdesk API
    url = f"{base_url}/incidents"
    print(f"\nTesting with TOPdesk API: {url}")
    
    try:
        response = requests.post(url, headers=headers, json=topdesk_payload, timeout=30)
        print(f"Status: {response.status_code}")
        print(f"Response: {response.text}")
        
        if response.status_code in [200, 201]:
            result = response.json()
            print(f"\n✅ SUCCESS! Incident created:")
            print(f"   Number: {result.get('number', 'N/A')}")
            print(f"   ID: {result.get('id', 'N/A')}")
        else:
            print(f"\n❌ FAILED: {response.status_code}")
            
    except Exception as e:
        print(f"\n❌ ERROR: {e}")

if __name__ == '__main__':
    print("MCP Server Test with 11Labs Payload Format")
    print("=" * 50)
    
    # Test MCP server
    test_mcp_server_with_llabs_payload()
    
    # Test direct TOPdesk API for comparison
    test_direct_topdesk_api()
    
    print("\n" + "=" * 50)
    print("Test completed!")
    print("If MCP fails but direct API works, the issue is in MCP server configuration."

In [ ]:
# Quick connectivity test
import requests
import json

def quick_test():
    print("🔧 Quick MCP connectivity test...")
    
    # Test the MCP endpoint
    mcp_url = 'https://matrosmcp.duckdns.org/topdesk/mcp'
    token = 'e3707c16425c14fa417e2384a12748c0c7c51dfdfd1714c58992215983f33257'
    
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    try:
        # Simple ping test
        response = requests.get(mcp_url, headers=headers, timeout=10)
        print(f"GET Status: {response.status_code}")
        print(f"Response: {response.text[:200]}...")
        
    except requests.exceptions.Timeout:
        print("❌ TIMEOUT - MCP server not responding")
    except requests.exceptions.ConnectionError:
        print("❌ CONNECTION ERROR - Cannot reach MCP server")
    except Exception as e:
        print(f"❌ ERROR: {e}")

quick_test()